# Finetuning Wave2Vec on Seismic Data

In [1]:
# WARNING: Conda environment manipulation (create/activate) often requires
# a kernel restart. Run the create/activate commands in separate cells.

# 1. Create a new, clean environment (Add -y)
# If you run this, you will need to restart your Jupyter kernel and select 'my_audio_env'.
%conda create -n my_audio_env python=3.12 -y

# 2. Activate the environment (If you are running in a terminal, not a notebook)
# In a notebook, this command often fails or requires installing ipykernel in the new env.
%conda activate my_audio_env

# 3. Install PyTorch and Torchaudio (Add -y)
%conda install pytorch torchaudio cpuonly -c pytorch -y

# 4. Install PyTables and HDF5 (Add -y)
%conda install pytables -c conda-forge -y

# 5. Install the remaining dependencies using pip (pip does not need -y)
!pip install pandas numpy transformers torchaudio

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.9.0
  latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=25.7.0



## Package Plan ##

  environment location: /Users/mabelwylie/miniconda3/envs/my_audio_env

  added / updated specs:
    - python=3.11


The following NEW packages will be INSTALLED:

  bzip2              pkgs/main/osx-64::bzip2-1.0.8-h6c40b1e_6 
  ca-certificates    pkgs/main/osx-64::ca-certificates-2025.7.15-hecd8cb5_0 
  expat              pkgs/main/osx-64::expat-2.7.1-h6d0c2b6_0 
  libcxx             pkgs/main/osx-64::libcxx-19.1.7-haebbb44_3 
  libffi             pkgs/main/osx-64::libffi-3.4.4-hecd8cb5_1 
  ncurses            pkgs/main/osx-64::ncurses-6.5-h923df54_0 
  openssl            pkgs/main/osx-64::openssl-3.0.17-hee2dfae_0 
  pip                pkgs/main/noarch::pi

In [1]:
# import required libraries
import pandas as pd
import numpy as np
import torchaudio
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor
import hdf5
import pytables

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'hdf5'

In [ ]:
# load and resample audio data using test data
eval_df=pd.read_pickle("../llaurenti_project/stead_test_subset.csv")
print("length eval_df:", len(eval_df))
eval_df.head()

length eval_df: 300


,index,E_channel,N_channel,Z_channel,back_azimuth_deg,coda_end_sample,network_code,p_arrival_sample,p_status,p_travel_sec,...,source_magnitude_author,source_magnitude_type,source_mechanism_strike_dip_rake,source_origin_time,source_origin_uncertainty_sec,trace_category,trace_name,trace_start_time,missing_time_from_event,source_magnitude_label
0,69180,"[-0.0, -0.006526834, -0.011599667, -0.03860825...","[-0.0, -0.0016655686, 0.017617673, 0.0618087, ...","[0.0, 0.0026812945, -0.0116409315, -0.01625718...",175.0,[[2172.0]],HV,802,autopicker,1.5,...,None,ml,None,2018-07-13 13:38:47.810,None,earthquake_local,RSDD.HV_20180713133840_EV,2018-07-13 13:38:41.290,-6.52,"[0, 0, 1, 0, 0]"
1,90101,"[-200.5755, -223.68773, -254.1443, -263.09805,...","[-25.060225, -34.886604, -54.364525, -68.77422...","[255.91516, 237.93314, 178.58626, 130.66376, 1...",271.700012,[[5238.0]],CI,900,manual,14.02,...,None,mb,None,2010-04-22 14:22:02.720,0.86,earthquake_local,BAR.CI_20100422142206_EV,2010-04-22 14:22:07.740,5.02,"[0, 0, 0, 1, 0]"
2,179676,"[2.440298, -2.5949564, -4.2758465, -4.798873, ...","[24.992334, 20.990742, 13.06829, 5.9852777, 3....","[-4.3666267, -6.9156466, -11.640782, -16.57282...",313.640015,[[3603.0]],AK,900,manual,12.74,...,None,ml,None,2018-07-12 23:34:52.545,1.15,earthquake_local,CHI.AK_20180712233456_EV,2018-07-12 23:34:56.285,3.74,"[0, 0, 1, 0, 0]"
3,36809,"[-0.0, -0.037904758, -0.087882325, -0.23007753...","[0.0, 0.08645568, 0.22473973, 0.58616436, 0.79...","[0.0, 0.0076630055, 0.039201356, 0.056555204, ...",98.400002,[[3044.0]],GS,900,manual,4.19,...,None,ml,None,2015-02-28 22:03:59.860,None,earthquake_local,KAN10.GS_20150228220354_EV,2015-02-28 22:03:55.050,-4.81,"[0, 1, 0, 0, 0]"
4,53115,"[550.6599, 596.43976, 617.6446, 610.97595, 595...","[-286.6528, -257.0198, -231.17693, -221.74454,...","[1.5144447, 64.00696, 125.89976, 132.23563, 88...",21.9,[[5900.0]],AV,900,manual,20.860001,...,None,mb,None,2012-01-04 17:24:31.700,0.69,earthquake_local,RDJH.AV_20120104172442_EV,2012-01-04 17:24:43.560,11.86,"[0, 0, 0, 0, 1]"


In [ ]:
# loads pretrained tokenizer and model
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# we first define a function to normalize the dataframe column
def normalize_df_column(df_column):
    """ This returns the normalized version of a df columns
    """
    # Stacks each array in the dataframe column into a single 2D numpy array
    dataset = np.vstack(df_column.values).astype(float) # Shape: (300, 6000)
    
    # Fix the data type mismatch while converting (use .float() for performance)
    src = torch.from_numpy(dataset).float()

    # it normalizes each event independently
    src_max = src.abs().max(dim=1)[0].reshape(-1,1)
    src_norm = src/src_max
    src = src_norm.unsqueeze(1) # Shape: (300, 1, 6000)
    
    return src # Shape: torch.Size([300, 1, 6000])

# --- Calling Code ---

# Ensure your Resampler is configured for Float
resampler = torchaudio.transforms.Resample(orig_freq=100, new_freq=16000)

normed_waveforms = normalize_df_column(eval_df["E_channel"])

# Perform resampling
resampled_waveforms = resampler(normed_waveforms)

NameError: name 'torchaudio' is not defined

In [ ]:
# check that normed_waveforms are normalized between -1 and 1
print("Min value after normalization:", normed_waveforms.min().item())
print("Max value after normalization:", normed_waveforms.max().item())
# check the shape of resampled_waveforms (expecting 6000*160=960000 samples)
print("Shape of resampled_waveforms:", resampled_waveforms.shape)

Min value after normalization: -1.0
Max value after normalization: 1.0
Shape of resampled_waveforms: torch.Size([300, 1, 960000])


In [ ]:
from datasets import load_dataset
# create a dataset class
class EQDataset(Dataset):
    def __init__(self, dataframe, tokenizer, resampler):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.resampler = resampler

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        waveform = self.dataframe.iloc[idx]['N_channel']
        # Resample the waveform
        waveform = self.resampler(waveform)
        input_values = self.tokenizer(waveform.squeeze().numpy(), return_tensors="pt", padding="longest").input_values
        return input_values.squeeze(0)

In [ ]:
data = EQDataset(eval_df, tokenizer, resampler)

In [ ]:
load_dataset('EQDataset', data_files={'test': '../llaurenti_project/stead_test_subset.csv'})

In [ ]:
# i want to load the first line of an hdf5 file into a pandas dataframe
pd.read_hdf('../raw_data/merge.hdf5',stop = 10)

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.